# Day 09. Exercise 01
# Gridsearch

## 0. Imports

In [1]:
import pandas as pd
import numpy as np

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

## 1. Preprocessing

1. Read the file [`day-of-week-not-scaled.csv`](https://drive.google.com/file/d/1AlGvsJDSzPT_70caausx8bFuupIEZkfh/view?usp=sharing). It is similar to the one from the previous exercise, but this time we did not scale continuous features (we are not going to use logreg anymore).
2. Using `train_test_split` with parameters `test_size=0.2`, `random_state=21` get `X_train`, `y_train`, `X_test`, `y_test`. Use the additional parameter `stratify`.

In [2]:
df = pd.read_csv('../data/dayofweek.csv')

X = df.drop('dayofweek', axis=1)
y = df['dayofweek']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21, stratify=y)

## 2. SVM gridsearch

1. Using `GridSearchCV` try different parameters of kernel (`linear`, `rbf`, `sigmoid`), C (`0.01`, `0.1`, `1`, `1.5`, `5`, `10`), gamma (`scale`, `auto`), class_weight (`balanced`, `None`) use `random_state=21` and `probability=True` and get the best combination of them in terms of accuracy.
2. Create a dataframe from the results of the gridsearch and sort it ascendingly by the `rank_test_score`. Check if there is a huge difference between different combinations (sometimes a simpler model may give a comparable result).

In [3]:
param_grid = {
    'kernel': ['linear', 'rbf', 'sigmoid'],
    'C': [0.01, 0.1, 1, 1.5, 5, 10],
    'gamma': ['scale', 'auto'],
    'class_weight': ['balanced', None]
}

svm = SVC(random_state=21, probability=True)
grid_search = GridSearchCV(svm, param_grid, cv=10, n_jobs=-1)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best Score:", best_score)

Best Parameters: {'C': 10, 'class_weight': 'balanced', 'gamma': 'scale', 'kernel': 'rbf'}
Best Score: 0.8678883360972914


In [4]:
from pandas import json_normalize

def get_gs_results(grid_search):
    results = pd.DataFrame(grid_search.cv_results_)
    params = json_normalize(results['params'])

    results_df = pd.concat([params, results[['mean_test_score', 'rank_test_score']]], axis=1)
    results_df.sort_values('rank_test_score', inplace=True)

    return results_df

In [5]:
get_gs_results(grid_search).head()

,C,class_weight,gamma,kernel,mean_test_score,rank_test_score
61,10.0,balanced,scale,rbf,0.867888,1
67,10.0,None,scale,rbf,0.867888,2
55,5.0,None,scale,rbf,0.856772,3
49,5.0,balanced,scale,rbf,0.847131,4
43,1.5,None,scale,rbf,0.825633,5


## 3. Decision tree

1. Using `GridSearchCV` try different parameters of `max_depth` (from `1` to `49`), `class_weight` (`balanced`, `None`) and `criterion` (`entropy` and `gini`) and get the best combination of them in terms of accuracy. Use `random_state=21`.
2. Create a dataframe from the results of the gridsearch and sort it ascendingly by the `rank_test_score`, check if there is a huge difference between different combinations (sometimes a simpler model may give a comparable result).

In [6]:
param_grid = {
    'max_depth': range(1, 50),
    'class_weight': ['balanced', None],
    'criterion': ['entropy', 'gini']
}

dt = DecisionTreeClassifier(random_state=21)

grid_search = GridSearchCV(estimator=dt, param_grid=param_grid, scoring='accuracy', cv=10)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best Score:", best_score)

Best Parameters: {'class_weight': None, 'criterion': 'entropy', 'max_depth': 23}
Best Score: 0.8842564953012715


In [7]:
get_gs_results(grid_search).head()

,class_weight,criterion,max_depth,mean_test_score,rank_test_score
120,None,entropy,23,0.884256,1
22,balanced,entropy,23,0.883521,2
141,None,entropy,44,0.883510,3
131,None,entropy,34,0.883510,3
132,None,entropy,35,0.883510,3


## 4. Random forest

1. Using `GridSearchCV` try different parameters of `n_estimators` (`5`, `10`, `50`, `100`), `max_depth` (from `1` to `49`), `class_weight` (`balanced`, `None`) and `criterion` (`entropy` and `gini`) and get the best combination of them in terms of accuracy. Use random_state=21.
2. Create a dataframe from the results of the gridsearch and sort it ascendengly by the `rank_test_score`, check if there is a huge difference between different combinations (sometimes a simpler model may give a comparable result).

In [8]:
param_grid = {
    'n_estimators': [5, 10, 50, 100],
    'max_depth': range(1, 50),
    'class_weight': ['balanced', None],
    'criterion': ['entropy', 'gini']
}

rf = RandomForestClassifier(random_state=21)

grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, scoring='accuracy', cv=10)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best Score:", best_score)

Best Parameters: {'class_weight': None, 'criterion': 'entropy', 'max_depth': 30, 'n_estimators': 100}
Best Score: 0.897584300718629


In [9]:
get_gs_results(grid_search).head()

,class_weight,criterion,max_depth,n_estimators,mean_test_score,rank_test_score
511,None,entropy,30,100,0.897584,1
535,None,entropy,36,100,0.895368,2
522,None,entropy,33,50,0.894627,3
506,None,entropy,29,50,0.894621,4
343,balanced,gini,37,100,0.894616,5


## 5. Progress bar

Gridsearch can be a quite long process and you may find yourself wondering when it will end.
1. Create a manual gridsearch for the same parameters values of random forest iterating through the list of the possible values and calculating `cross_val_score` for each combination. Try to increase `n_jobs`. The value `cv` for `cross_val_score` is 5.
2. Track the progress using the library `tqdm.notebook`.
3. Create a dataframe from the results of the gridsearch with the columns corresponding to the names of the parameters and `mean_accuracy` and `std_accuracy`.
4. Sort it descendingly by the `mean_accuracy`, check if there is a huge difference between different combinations (sometimes a simpler model may give a comparable result).

In [4]:
param_grid = {
    'n_estimators': [5, 10, 50, 100],
    'max_depth': range(1, 50),
    'class_weight': ['balanced', None],
    'criterion': ['entropy', 'gini']
}

results = []

for n_estimators in tqdm(param_grid['n_estimators'], desc='n_estimators'):
    for max_depth in tqdm(param_grid['max_depth'], desc='max_depth', leave=False):
        for class_weight in tqdm(param_grid['class_weight'], desc='class_weight', leave=False):
            for criterion in tqdm(param_grid['criterion'], desc='criterion', leave=False):
                rf = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, class_weight=class_weight, criterion=criterion, random_state=21)
                scores = cross_val_score(rf, X_train, y_train, cv=5, n_jobs=-1)
                mean_score = scores.mean()
                std_score = scores.std()
                results.append({
                    'n_estimators': n_estimators,
                    'max_depth': max_depth,
                    'class_weight': class_weight,
                    'criterion': criterion,
                    'mean_accuracy': mean_score,
                    'std_accuracy': std_score
                })


n_estimators:   0%|          | 0/4 [00:00<?, ?it/s]

max_depth:   0%|          | 0/49 [00:00<?, ?it/s]

class_weight:   0%|          | 0/2 [00:00<?, ?it/s]

criterion:   0%|          | 0/2 [00:00<?, ?it/s]

criterion:   0%|          | 0/2 [00:00<?, ?it/s]

...

criterion:   0%|          | 0/2 [00:00<?, ?it/s]

criterion:   0%|          | 0/2 [00:00<?, ?it/s]

class_weight:   0%|          | 0/2 [00:00<?, ?it/s]

criterion:   0%|          | 0/2 [00:00<?, ?it/s]

criterion:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
results_df = pd.DataFrame(results)
results_df = results_df.sort_values(by='mean_accuracy', ascending=False)

results_df.head()

,n_estimators,max_depth,class_weight,criterion,mean_accuracy,std_accuracy
545,50,39,balanced,gini,0.882029,0.026083
529,50,35,balanced,gini,0.881291,0.024154
549,50,40,balanced,gini,0.881289,0.021882
557,50,42,balanced,gini,0.881289,0.021882
537,50,37,balanced,gini,0.880551,0.022776


## 6. Predictions

1. Choose the best model and use it to make predictions for the test dataset.
2. Calculate the final accuracy.

In [9]:
best_model = RandomForestClassifier(n_estimators=50, max_depth=39, class_weight='balanced', criterion='gini', random_state=21)

best_model.fit(X_train, y_train)

y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Final Accuracy:", accuracy)

Final Accuracy: 0.8994082840236687
